In [115]:
import requests
import re

In [116]:
class SPSIOP(object):
    _url_spsiop = r'https://www.marketwatch.com/investing/index/spsiop?countrycode=xx'
    _url_162411 = r'http://www.fsfund.com/funds/162411/index.shtml'
    
    def __init__(self):
        self._r_spsiop = requests.get(self._url_spsiop)
    
    def get_close_price(self):
        pattern = r'(\d+,)*\d+(\.\d*)?'
        item = "<meta name=\"price\" content=\"(%s)\">" % pattern
        regex = re.compile(item)
        serchObj = regex.search(self._r_spsiop.text)
        str_price = serchObj.group(1)
        str_price = re.sub(',', '', str_price)
        return float(str_price)
    
    def get_open_price(self):
        pattern = r'(\d+,)*\d+(\.\d*)?'
        item = "<span class=\"open-value\">(%s)</span></div>" % pattern
        regex = re.compile(item)
        serchObj = regex.search(self._r_spsiop.text)
        str_price_change = serchObj.group(1)
        str_price_change = re.sub(',', '', str_price_change)
        return float(str_price_change)
    
    def get_change_percent(self):
        close_price = self.get_close_price()
        open_price = self.get_open_price()
        price_change = close_price - open_price
        change_percent = price_change / open_price
        print("open_price: %s, close_price: %s, price_change: %.2f, change_percent: %.2f%%" %
             (open_price, close_price, price_change, change_percent*100))
        return change_percent

In [117]:
sp = SPSIOP()
sp.get_change_percent()

open_price: 4601.35, close_price: 4655.46, price_change: 54.11, change_percent: 1.18%


0.011759592293565947

In [5]:
sp.get_change_percent()

open_price: 4601.35, close_price: 4655.46, price_change: 54.11, change_percent: 1.18%


0.011759592293565947

In [111]:
import re
import datetime

def parse_day(date):
    day_pattern = r'(\d+)-(0?[1-9]|1[0-2])-([12][0-9]|3[01]|0?[1-9])'
    searchObj = re.search(day_pattern, date)
    year, month, day = searchObj.groups()
    return int(year), int(month), int(day)

def get_int_day(date):
    year, month, day = parse_day(date)
    return year*100*100 + month*100 + day

def next_nmonth_first_day(date, n=1):
    year, month, day = parse_day(date)
    next_moth = (month + n - 1) % 12 + 1
    year += (month + n - 1) // 12
    return '%s-%02d-%02d' % (year, next_moth, 1)

def next_nday(date, n=1):
    year, month, day = parse_day(date)
    ndate = datetime.date(year, month, day) + datetime.timedelta(days=n)
    return ndate.strftime("%Y-%m-%d")

def GE(date1, date2):
    """
    params:
    dau1, day2 (xxxx-xx-xx)
    return day1 is equal or grate than day2
    """
    d1 = get_int_day(date1)
    d2 = get_int_day(date2)
    return d1 >= d2
# GE('2019-12-01', '2019-12-01')

In [113]:
import time
import h5py
import numpy as np
import re
from bs4 import BeautifulSoup

url_162411 = r'http://www.fsfund.com/hbdynamic/selHistoryNavX.do'
user_agent = r'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'

headers = {
    'Accept' : 'text/html, */*; q=0.01',
    'Accept-Encoding' : 'gzip, deflate',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Cache-Control' : 'max-age=0',
    'Host' : 'www.fsfund.com',
    'Proxy-Connection' : 'keep-alive',
    'X-Requested-With': 'XMLHttpRequest',
    'User-Agent' : user_agent,
}
 
class HBYQ(object):
    _init_day = '2011-10-21'
    
    def __init(self):
        pass
    
    def get_row_datas(self, start_day, end_day):
        params = {
            'vc_fundcode' : '162411',
            'i_fundstyle': '201008',
            'quarterdate1': start_day,
            'quarterdate2': end_day,
            'paging': 'n',
        }
        r = requests.post(_url_162411,headers = headers, params = params)
        
        return r.content.decode('utf-8')
    
    def parse_row(self, content):
        soup = BeautifulSoup(content, 'lxml')
        trs = soup.find_all('tr')
        datas = []
        for tr in trs:
            data = tr.get_text().split()
            datas.append(data)
        return datas
    
    def get_datas(self, start_day, end_day):
        rows = self.get_row_datas(start_day, end_day)
        datas = self.parse_row(rows)
        return datas
    
    def get_all_data(self):
        today = time.strftime("%Y-%m-%d", time.localtime())
        start_day = self._init_day
        end_day = self._init_day
        while GE(today, start_day):
            print(self.get_datas(start_day, end_day))
            start_day = next_nday(end_day, 1)

In [114]:
hbyq = HBYQ()
hbyq.get_all_data()

[['2012-01-19', '0.985', '0.985', '-0.61%'], ['2012-01-18', '0.991', '0.991', '1.64%'], ['2012-01-17', '0.975', '0.975', '0.52%'], ['2012-01-13', '0.969', '0.969', '-1.22%'], ['2012-01-12', '0.981', '0.981', '-0.51%'], ['2012-01-11', '0.986', '0.986', '-1.30%'], ['2012-01-10', '0.999', '0.999', '0.60%'], ['2012-01-09', '0.993', '0.993', '-0.30%'], ['2012-01-06', '0.996', '0.996', '-0.50%'], ['2012-01-05', '1.001', '1.001', '0.00%'], ['2012-01-04', '1.001', '1.001', '2.35%'], ['2011-12-30', '0.978', '0.978', '-0.20%'], ['2011-12-29', '0.980', '0.980', '0.72%'], ['2011-12-28', '0.973', '0.973', '-1.32%'], ['2011-12-27', '0.986', '0.986', '0.41%'], ['2011-12-23', '0.982', '0.982', '0.00%'], ['2011-12-22', '0.982', '0.982', '0.61%'], ['2011-12-21', '0.976', '0.976', '0.51%'], ['2011-12-20', '0.971', '0.971', '2.32%'], ['2011-12-19', '0.949', '0.949', '-1.45%'], ['2011-12-16', '0.963', '0.963', '0.84%'], ['2011-12-15', '0.955', '0.955', '0.00%'], ['2011-12-14', '0.955', '0.955', '-1.85%'], 

[['2012-10-18', '1.000', '1.000', '-0.20%'], ['2012-10-17', '1.002', '1.002', '1.21%'], ['2012-10-16', '0.990', '0.990', '1.75%'], ['2012-10-15', '0.973', '0.973', '-0.31%'], ['2012-10-12', '0.976', '0.976', '-1.11%'], ['2012-10-11', '0.987', '0.987', '1.13%'], ['2012-10-10', '0.976', '0.976', '-1.51%'], ['2012-10-09', '0.991', '0.991', '1.54%'], ['2012-09-28', '0.981', '0.981', '-0.30%'], ['2012-09-27', '0.984', '0.984', '2.18%'], ['2012-09-26', '0.963', '0.963', '-1.63%'], ['2012-09-25', '0.979', '0.979', '-1.41%'], ['2012-09-24', '0.993', '0.993', '-1.19%'], ['2012-09-21', '1.005', '1.005', '0.60%'], ['2012-09-20', '0.999', '0.999', '0.20%'], ['2012-09-19', '0.997', '0.997', '-0.60%'], ['2012-09-18', '1.003', '1.003', '-1.76%'], ['2012-09-17', '1.021', '1.021', '-1.54%'], ['2012-09-14', '1.037', '1.037', '2.37%'], ['2012-09-13', '1.013', '1.013', '2.01%'], ['2012-09-12', '0.993', '0.993', '0.51%'], ['2012-09-11', '0.988', '0.988', '1.96%'], ['2012-09-10', '0.969', '0.969', '-0.92%']

[['2013-07-18', '1.051', '1.051', '1.64%'], ['2013-07-17', '1.034', '1.034', '0.78%'], ['2013-07-16', '1.026', '1.026', '-0.10%'], ['2013-07-15', '1.027', '1.027', '-0.68%'], ['2013-07-12', '1.034', '1.034', '0.00%'], ['2013-07-11', '1.034', '1.034', '0.88%'], ['2013-07-10', '1.025', '1.025', '-0.10%'], ['2013-07-09', '1.026', '1.026', '0.88%'], ['2013-07-08', '1.017', '1.017', '0.30%'], ['2013-07-05', '1.014', '1.014', '1.50%'], ['2013-07-03', '1.000', '1.000', '0.50%'], ['2013-07-02', '0.995', '0.995', '0.20%'], ['2013-07-01', '0.993', '0.993', '0.61%'], ['2013-06-28', '0.987', '0.987', '-0.20%'], ['2013-06-27', '0.989', '0.989', '-0.10%'], ['2013-06-26', '0.990', '0.990', '0.20%'], ['2013-06-25', '0.988', '0.988', '1.65%'], ['2013-06-24', '0.972', '0.972', '-1.32%'], ['2013-06-21', '0.985', '0.985', '-0.10%'], ['2013-06-20', '0.986', '0.986', '-3.80%'], ['2013-06-19', '1.025', '1.025', '-0.68%'], ['2013-06-18', '1.032', '1.032', '0.88%'], ['2013-06-17', '1.023', '1.023', '1.69%'], [

[['2014-04-17', '1.256', '1.256', '1.45%'], ['2014-04-16', '1.238', '1.238', '1.56%'], ['2014-04-15', '1.219', '1.219', '1.75%'], ['2014-04-14', '1.198', '1.198', '2.04%'], ['2014-04-11', '1.174', '1.174', '-0.51%'], ['2014-04-10', '1.180', '1.180', '-1.42%'], ['2014-04-09', '1.197', '1.197', '0.93%'], ['2014-04-08', '1.186', '1.186', '-0.42%'], ['2014-04-04', '1.191', '1.191', '-0.92%'], ['2014-04-03', '1.202', '1.202', '0.50%'], ['2014-04-02', '1.196', '1.196', '1.18%'], ['2014-04-01', '1.182', '1.182', '0.60%'], ['2014-03-31', '1.175', '1.175', '0.17%'], ['2014-03-28', '1.173', '1.173', '1.47%'], ['2014-03-27', '1.156', '1.156', '1.58%'], ['2014-03-26', '1.138', '1.138', '-1.22%'], ['2014-03-25', '1.152', '1.152', '0.44%'], ['2014-03-24', '1.147', '1.147', '-0.69%'], ['2014-03-21', '1.155', '1.155', '0.52%'], ['2014-03-20', '1.149', '1.149', '0.97%'], ['2014-03-19', '1.138', '1.138', '-0.70%'], ['2014-03-18', '1.146', '1.146', '1.87%'], ['2014-03-17', '1.125', '1.125', '-0.09%'], ['

[['2015-04-16', '0.831', '0.831', '-0.72%'], ['2015-04-15', '0.837', '0.837', '3.46%'], ['2015-04-14', '0.809', '0.809', '3.06%'], ['2015-04-13', '0.785', '0.785', '-0.76%'], ['2015-04-10', '0.791', '0.791', '0.51%'], ['2015-04-09', '0.787', '0.787', '1.81%'], ['2015-04-08', '0.773', '0.773', '-2.77%'], ['2015-04-07', '0.795', '0.795', '3.65%'], ['2015-04-03', '0.767', '0.767', '0.00%'], ['2015-04-02', '0.767', '0.767', '1.05%'], ['2015-04-01', '0.759', '0.759', '1.34%'], ['2015-03-31', '0.749', '0.749', '-0.13%'], ['2015-03-30', '0.750', '0.750', '1.76%'], ['2015-03-27', '0.737', '0.737', '-2.12%'], ['2015-03-26', '0.753', '0.753', '1.07%'], ['2015-03-25', '0.745', '0.745', '1.92%'], ['2015-03-24', '0.731', '0.731', '0.83%'], ['2015-03-23', '0.725', '0.725', '-1.36%'], ['2015-03-20', '0.735', '0.735', '0.96%'], ['2015-03-19', '0.728', '0.728', '-2.54%'], ['2015-03-18', '0.747', '0.747', '4.33%'], ['2015-03-17', '0.716', '0.716', '-1.24%'], ['2015-03-16', '0.725', '0.725', '0.14%'], ['

[['2016-04-14', '0.541', '0.541', '0.56%'], ['2016-04-13', '0.538', '0.538', '-1.10%'], ['2016-04-12', '0.544', '0.544', '6.46%'], ['2016-04-11', '0.511', '0.511', '-0.20%'], ['2016-04-08', '0.512', '0.512', '3.64%'], ['2016-04-07', '0.494', '0.494', '0.00%'], ['2016-04-06', '0.494', '0.494', '3.56%'], ['2016-04-05', '0.477', '0.477', '-1.85%'], ['2016-04-01', '0.486', '0.486', '-1.82%'], ['2016-03-31', '0.495', '0.495', '1.23%'], ['2016-03-30', '0.489', '0.489', '-0.41%'], ['2016-03-29', '0.491', '0.491', '1.03%'], ['2016-03-28', '0.486', '0.486', '-1.02%'], ['2016-03-25', '0.491', '0.491', '0.20%'], ['2016-03-24', '0.490', '0.490', '1.66%'], ['2016-03-23', '0.482', '0.482', '-4.74%'], ['2016-03-22', '0.506', '0.506', '0.40%'], ['2016-03-21', '0.504', '0.504', '0.20%'], ['2016-03-18', '0.503', '0.503', '-0.79%'], ['2016-03-17', '0.507', '0.507', '1.40%'], ['2016-03-16', '0.500', '0.500', '3.95%'], ['2016-03-15', '0.481', '0.481', '0.21%'], ['2016-03-14', '0.480', '0.480', '-1.64%'], [

[['2017-04-13', '0.622', '0.622', '-2.81%'], ['2017-04-12', '0.640', '0.640', '-1.69%'], ['2017-04-11', '0.651', '0.651', '0.00%'], ['2017-04-10', '0.651', '0.651', '1.72%'], ['2017-04-07', '0.640', '0.640', '-0.62%'], ['2017-04-06', '0.644', '0.644', '1.42%'], ['2017-04-05', '0.635', '0.635', '-1.40%'], ['2017-03-31', '0.644', '0.644', '1.42%'], ['2017-03-30', '0.635', '0.635', '-0.94%'], ['2017-03-29', '0.641', '0.641', '2.72%'], ['2017-03-28', '0.624', '0.624', '2.63%'], ['2017-03-27', '0.608', '0.608', '0.33%'], ['2017-03-24', '0.606', '0.606', '-0.16%'], ['2017-03-23', '0.607', '0.607', '-0.33%'], ['2017-03-22', '0.609', '0.609', '-1.14%'], ['2017-03-21', '0.616', '0.616', '-2.07%'], ['2017-03-20', '0.629', '0.629', '0.16%'], ['2017-03-17', '0.628', '0.628', '-0.16%'], ['2017-03-16', '0.629', '0.629', '-1.26%'], ['2017-03-15', '0.637', '0.637', '2.91%'], ['2017-03-14', '0.619', '0.619', '-0.96%'], ['2017-03-13', '0.625', '0.625', '0.48%'], ['2017-03-10', '0.622', '0.622', '-0.48%'

[['2018-01-11', '0.635', '0.635', '2.09%'], ['2018-01-10', '0.622', '0.622', '-0.16%'], ['2018-01-09', '0.623', '0.623', '0.16%'], ['2018-01-08', '0.622', '0.622', '0.32%'], ['2018-01-05', '0.620', '0.620', '-0.80%'], ['2018-01-04', '0.625', '0.625', '0.64%'], ['2018-01-03', '0.621', '0.621', '0.81%'], ['2018-01-02', '0.616', '0.616', '1.99%'], ['2017-12-29', '0.604', '0.604', '-1.15%'], ['2017-12-28', '0.611', '0.611', '0.83%'], ['2017-12-27', '0.606', '0.606', '-0.82%'], ['2017-12-26', '0.611', '0.611', '1.66%'], ['2017-12-25', '0.601', '0.601', '-0.17%'], ['2017-12-22', '0.602', '0.602', '0.50%'], ['2017-12-21', '0.599', '0.599', '2.22%'], ['2017-12-20', '0.586', '0.586', '2.27%'], ['2017-12-19', '0.573', '0.573', '0.00%'], ['2017-12-18', '0.573', '0.573', '1.96%'], ['2017-12-15', '0.562', '0.562', '-0.71%'], ['2017-12-14', '0.566', '0.566', '-1.39%'], ['2017-12-13', '0.574', '0.574', '-0.69%'], ['2017-12-12', '0.578', '0.578', '-1.53%'], ['2017-12-11', '0.587', '0.587', '1.03%'], [

[['2018-10-11', '0.694', '0.694', '-2.80%'], ['2018-10-10', '0.714', '0.714', '-4.55%'], ['2018-10-09', '0.748', '0.748', '1.36%'], ['2018-10-08', '0.738', '0.738', '0.41%'], ['2018-09-28', '0.735', '0.735', '0.55%'], ['2018-09-27', '0.731', '0.731', '0.97%'], ['2018-09-26', '0.724', '0.724', '-1.63%'], ['2018-09-25', '0.736', '0.736', '2.79%'], ['2018-09-21', '0.716', '0.716', '0.56%'], ['2018-09-20', '0.712', '0.712', '-0.28%'], ['2018-09-19', '0.714', '0.714', '0.99%'], ['2018-09-18', '0.707', '0.707', '1.43%'], ['2018-09-17', '0.697', '0.697', '0.00%'], ['2018-09-14', '0.697', '0.697', '0.29%'], ['2018-09-13', '0.695', '0.695', '-0.71%'], ['2018-09-12', '0.700', '0.700', '1.45%'], ['2018-09-11', '0.690', '0.690', '1.92%'], ['2018-09-10', '0.677', '0.677', '0.45%'], ['2018-09-07', '0.674', '0.674', '-0.74%'], ['2018-09-06', '0.679', '0.679', '-3.14%'], ['2018-09-05', '0.701', '0.701', '-0.43%'], ['2018-09-04', '0.704', '0.704', '-1.40%'], ['2018-09-03', '0.714', '0.714', '0.14%'], [

9

In [42]:
day_pattern = r'(\d+)-(0?[1-9]|1[0-2])-([12][0-9]|3[01]|0?[1-9])'
day_regex = re.compile(day_pattern)
price_pattern = r'(\d+,)*\d+(\.\d*)?'
price_regex = re.compile(price_pattern)


#     update_time = day_regex.search(ui[2]).group(0)
#     price = price_regex.search(ui[3]).group(0)
#     price = re.sub(',', '', price)
    
#     print(update_time, price)


['2019-01-24', '0.511', '0.511', '0.39%']
['2019-01-23', '0.509', '0.509', '-1.36%']
['2019-01-22', '0.516', '0.516', '-3.91%']
['2019-01-21', '0.537', '0.537', '0.19%']
['2019-01-18', '0.536', '0.536', '1.90%']
['2019-01-17', '0.526', '0.526', '0.57%']
['2019-01-16', '0.523', '0.523', '0.19%']
['2019-01-15', '0.522', '0.522', '0.97%']
['2019-01-14', '0.517', '0.517', '-0.19%']
['2019-01-11', '0.518', '0.518', '-1.52%']
['2019-01-10', '0.526', '0.526', '-0.19%']
['2019-01-09', '0.527', '0.527', '2.53%']
['2019-01-08', '0.514', '0.514', '0.59%']
['2019-01-07', '0.511', '0.511', '3.23%']
['2019-01-04', '0.495', '0.495', '5.32%']
['2019-01-03', '0.470', '0.470', '-0.42%']
['2019-01-02', '0.472', '0.472', '3.06%']


In [100]:
import datetime

In [110]:
today = datetime.date.today().strftime("%Y-%m-%d")
print(today)
year, month, day = parse_day(today)
datetime.date(year, month, day) + datetime.timedelta(days=34)

2019-01-26


datetime.date(2019, 3, 1)